In [1]:
import pandas as pd
import numpy as np
import pandas_profiling as pdp

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics, svm
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor, BaggingRegressor


# display plots in the notebook
%matplotlib inline
# increase default figure and font sizes for easier viewing
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 12

In [26]:
#reading in test file
test = pd.read_csv("../assets/test.csv")
test["Date"] = pd.to_datetime(test["Date"], infer_datetime_format=True)

#Creating "Parent Trap" column that will indicate all traps associated with a parent trap (e.g., T002A would be T002)
test["Parent_Trap"] = test["Trap"].str[:4]


In [27]:
test

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,Parent_Trap
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,T002
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,T002
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,T002
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,T002
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,T002
5,6,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TARSALIS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,T002
6,7,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",UNSPECIFIED CULEX,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,T002
7,8,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX ERRATICUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,T002
8,9,2008-06-11,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX PIPIENS/RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,T007
9,10,2008-06-11,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,T007


In [28]:
#reading in cleaned train data for purposes of merging generated predictor columns onto test set
#train = pd.read_csv("../assets/clean_train.csv", index_col=0)
train = pd.read_csv("../assets/clean_train_2.csv", index_col=0)


train["Date"] = pd.to_datetime(train["Date"], infer_datetime_format=True)

In [29]:
train.head()

,Date,Species,Trap,Latitude,Longitude,NumMosquitos,WnvPresent,Year,Parent_Trap,Is_Satellite,...,Trap_Species_Mos_75percent,Trap_Mos_Mean,Trap_Mos_Median,Trap_Mos_25percent,Trap_Mos_75percent,Species_Mos_Mean,Species_Mos_Median,Species_Mos_25percent,Species_Mos_75percent,Species_Obs_Proportion
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.95469,-87.800991,1.0,0.0,2007.0,T002,0.0,...,29.0,25.238095,10.0,3.0,24.0,17.320439,5.0,2.0,14.0,0.490732
1,2007-06-05,CULEX PIPIENS/RESTUANS,T002,41.95469,-87.800991,3.0,0.0,2007.0,T002,0.0,...,29.0,25.238095,10.0,3.0,24.0,17.320439,5.0,2.0,14.0,0.490732
2,2007-06-26,CULEX PIPIENS/RESTUANS,T002,41.95469,-87.800991,1.0,0.0,2007.0,T002,0.0,...,29.0,25.238095,10.0,3.0,24.0,17.320439,5.0,2.0,14.0,0.490732
3,2007-06-29,CULEX PIPIENS/RESTUANS,T002,41.95469,-87.800991,2.0,0.0,2007.0,T002,0.0,...,29.0,25.238095,10.0,3.0,24.0,17.320439,5.0,2.0,14.0,0.490732
4,2007-07-02,CULEX PIPIENS/RESTUANS,T002,41.95469,-87.800991,3.0,0.0,2007.0,T002,0.0,...,29.0,25.238095,10.0,3.0,24.0,17.320439,5.0,2.0,14.0,0.490732


In [30]:
#isolating and merging Mos_WNV_Prob variable from train into test set

mos_wnv_prob = train.groupby(["Species", "Mos_WNV_Prob"]).size()
mos_wnv_prob = pd.DataFrame(mos_wnv_prob)
mos_wnv_prob.reset_index(inplace=True)

#since we'll need to inpute Mos_WNV_Prob for "UNSPECIFIED CULEX", we'll take the proportion of total WNV detections
#in the train data divided by the total number of observations - essentially, the probability that any trap with 
#mosquitos of any type had WNV.

unspecified_wnv = train["WnvPresent"].sum() / train["WnvPresent"].count()

#Merging Mos_WNV_Prob onto test data.
test = pd.merge(test, mos_wnv_prob[["Species", "Mos_WNV_Prob"]],how="left", left_on='Species', right_on='Species')

#inputting averaged wnv probability into NaN values following merge
test["Mos_WNV_Prob"].fillna(value=unspecified_wnv, inplace=True)

test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,Parent_Trap,Mos_WNV_Prob
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,T002,0.058808
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,T002,0.020059
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,T002,0.094359
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,T002,0.000000
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,T002,0.000000


In [31]:
# Adding trap-specific variables from train set onto test set
trap_wnv_prob = train.groupby(['Trap', "Num_Years_Trap_WNV_Detection", "Max_One_Year_Trap_WNV_Detections",
                               "Total_Train_Trap_WNV_Detections", "Trap_WNV_Prob"]).size()
trap_wnv_prob = pd.DataFrame(trap_wnv_prob)
trap_wnv_prob.reset_index(inplace=True)

test = pd.merge(test, trap_wnv_prob[["Trap", "Num_Years_Trap_WNV_Detection", "Max_One_Year_Trap_WNV_Detections",
                                    "Total_Train_Trap_WNV_Detections", "Trap_WNV_Prob"]],how="left", 
                left_on=["Trap"], right_on=["Trap"])

#9905 null values - mostly from sattelite traps.  I'll scale down trap_wnv_prob to the parent traps of the missing 
#sattelites and do another merge on "Parent Trap" to fill in most of the remaining NaNs.

#identifying traps in test with NaN values and finding corresponding parent_trap values for imputation
test_missing_trap = test[test["Trap_WNV_Prob"].isnull()]
missing_traps = list(test_missing_trap["Parent_Trap"].unique())
missing_traps = trap_wnv_prob[trap_wnv_prob["Trap"].isin(missing_traps)]
missing_traps

#removing test_missing_trap rows from test so updated rows can be concatenated to test and easily resorted into the 
#right order
test = test[~test.isin(test_missing_trap)].dropna(how='all')

#dropping NaN columns from original merge attempt, and adding new columns based on parent trap id
test_missing_trap.dropna(axis=1, inplace=True)
test_missing_trap = pd.merge(test_missing_trap, missing_traps[["Trap", "Num_Years_Trap_WNV_Detection", 
                                                               "Max_One_Year_Trap_WNV_Detections",
                                                               "Total_Train_Trap_WNV_Detections", 
                                                               "Trap_WNV_Prob"]],how="left", left_on=["Parent_Trap"], 
                             right_on=["Trap"])

#dropping added trap_y column, and renaming trap_x to original trap
test_missing_trap.drop("Trap_y", axis=1, inplace=True)
test_missing_trap.rename(columns={'Trap_x':'Trap'}, inplace=True)

#concatenating original missing trap data for sattelite traps on test data
test = pd.concat([test, test_missing_trap], ignore_index=True, axis=0)

#Still one remaining trap (T234) with no values for the trap values above. For sake of ease, I'm going to input these
#with averages from every columns
test["Max_One_Year_Trap_WNV_Detections"].fillna(test["Max_One_Year_Trap_WNV_Detections"].mean(), inplace=True)
test["Num_Years_Trap_WNV_Detection"].fillna(test["Num_Years_Trap_WNV_Detection"].mean(), inplace=True)
test["Total_Train_Trap_WNV_Detections"].fillna(test["Total_Train_Trap_WNV_Detections"].mean(), inplace=True)
test["Trap_WNV_Prob"].fillna(test["Trap_WNV_Prob"].mean(), inplace=True)

test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,Parent_Trap,Mos_WNV_Prob,Num_Years_Trap_WNV_Detection,Max_One_Year_Trap_WNV_Detections,Total_Train_Trap_WNV_Detections,Trap_WNV_Prob
0,1.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9.0,T002,0.058808,4.0,7.0,15.0,0.102041
1,2.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9.0,T002,0.020059,4.0,7.0,15.0,0.102041
2,3.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9.0,T002,0.094359,4.0,7.0,15.0,0.102041
3,4.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9.0,T002,0.000000,4.0,7.0,15.0,0.102041
4,5.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9.0,T002,0.000000,4.0,7.0,15.0,0.102041


In [32]:
#isolating and merging Trap_Species_WNV_Prob variable from train into test set
trap_mos_wnv_prob = train.groupby(['Trap',"Species", "Trap_Species_WNV_Prob"]).size()
trap_mos_wnv_prob = pd.DataFrame(trap_mos_wnv_prob)
trap_mos_wnv_prob.reset_index(inplace=True)
trap_mos_wnv_prob

test = pd.merge(test, trap_mos_wnv_prob[["Trap","Species", "Trap_Species_WNV_Prob"]],how="left", 
                left_on=["Trap",'Species'], right_on=["Trap",'Species'])

#Lots of NaNs left here (63202) - how to inpute? Values are NaN if species was never see in trap before.  
#Perhaps take an average of the Mos_WNV_Prob and Trap_WNV Prob?
test["Trap_Species_WNV_Prob"].fillna((test["Mos_WNV_Prob"] + test["Trap_WNV_Prob"])/2, inplace=True)

test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,Parent_Trap,Mos_WNV_Prob,Num_Years_Trap_WNV_Detection,Max_One_Year_Trap_WNV_Detections,Total_Train_Trap_WNV_Detections,Trap_WNV_Prob,Trap_Species_WNV_Prob
0,1.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9.0,T002,0.058808,4.0,7.0,15.0,0.102041,0.142857
1,2.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9.0,T002,0.020059,4.0,7.0,15.0,0.102041,0.000000
2,3.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9.0,T002,0.094359,4.0,7.0,15.0,0.102041,0.162162
3,4.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9.0,T002,0.000000,4.0,7.0,15.0,0.102041,0.000000
4,5.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9.0,T002,0.000000,4.0,7.0,15.0,0.102041,0.000000


In [33]:
#inputting 'Species_Mos_Mean', 'Species_Mos_Median', 'Species_Mos_25percent','Species_Mos_75percent', 'Species_Obs_Proportion'
mos_spec_nums = train.groupby(["Species", 'Species_Mos_Mean', 'Species_Mos_Median', 'Species_Mos_25percent',
                              'Species_Mos_75percent', 'Species_Obs_Proportion']).size()
mos_spec_nums = pd.DataFrame(mos_spec_nums)
mos_spec_nums.reset_index(inplace=True)
mos_spec_nums

#since we'll need to inpute each of these variables for "UNSPECIFIED CULEX", we'll calculate the values for these 
#variables across all species of mosquitos and inpute them as such

#for unspecified cullex, just taking mean, median, etc. of all mosquitos  across all train observations
species_mos_mean = train.NumMosquitos.mean()
species_mos_median = train.NumMosquitos.median()
species_mos_25percent = train.NumMosquitos.quantile(q=.25)
species_mos_75percent = train.NumMosquitos.quantile(q=.75)

#averging the proportions of each species of mosquito for imputation on unspecified cullex
species_mos_proportion = mos_spec_nums["Species_Obs_Proportion"].mean()

#Merging Mos_WNV_Prob onto test data.
test = pd.merge(test, mos_spec_nums[["Species", 'Species_Mos_Mean', 'Species_Mos_Median', 'Species_Mos_25percent',
                              'Species_Mos_75percent', 'Species_Obs_Proportion']],how="left", 
                left_on='Species', right_on='Species')

#inputting averaged wnv probability into NaN values following merge
test["Species_Mos_Mean"].fillna(value=species_mos_mean, inplace=True)
test["Species_Mos_Median"].fillna(value=species_mos_median, inplace=True)
test["Species_Mos_25percent"].fillna(value=species_mos_25percent, inplace=True)
test["Species_Mos_75percent"].fillna(value=species_mos_75percent, inplace=True)
test["Species_Obs_Proportion"].fillna(value=species_mos_proportion, inplace=True)

test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,...,Num_Years_Trap_WNV_Detection,Max_One_Year_Trap_WNV_Detections,Total_Train_Trap_WNV_Detections,Trap_WNV_Prob,Trap_Species_WNV_Prob,Species_Mos_Mean,Species_Mos_Median,Species_Mos_25percent,Species_Mos_75percent,Species_Obs_Proportion
0,1.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,4.0,7.0,15.0,0.102041,0.142857,17.320439,5.0,2.0,14.0,0.490732
1,2.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,4.0,7.0,15.0,0.102041,0.000000,9.791475,3.0,1.0,8.0,0.173513
2,3.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,4.0,7.0,15.0,0.102041,0.162162,22.908205,4.0,2.0,10.0,0.330801
3,4.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,4.0,7.0,15.0,0.102041,0.000000,1.746988,1.0,1.0,2.0,0.001074
4,5.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,4.0,7.0,15.0,0.102041,0.000000,2.361111,1.0,1.0,2.0,0.003777


In [34]:
#inputting 'Trap_Mos_Mean', 'Trap_Mos_Median', 'Trap_Mos_25percent', 'Trap_Mos_75percent',

mos_trap_nums = train.groupby(["Trap", 'Trap_Mos_Mean', 'Trap_Mos_Median', 'Trap_Mos_25percent', 
                               'Trap_Mos_75percent']).size()
mos_trap_nums = pd.DataFrame(mos_trap_nums)
mos_trap_nums.reset_index(inplace=True)
mos_trap_nums

#Merging new variables onto test data.
test = pd.merge(test, mos_trap_nums[["Trap", 'Trap_Mos_Mean', 'Trap_Mos_Median', 'Trap_Mos_25percent', 
                               'Trap_Mos_75percent']],how="left", 
                left_on='Trap', right_on='Trap')


#identifying traps in test with NaN values and finding corresponding parent_trap values for imputation
test_missing_trap = test[test["Trap_Mos_75percent"].isnull()]
missing_traps = list(test_missing_trap["Parent_Trap"].unique())
missing_traps = mos_trap_nums[mos_trap_nums["Trap"].isin(missing_traps)]
missing_traps

#removing test_missing_trap rows from test so updated rows can be concatenated to test and easily resorted into the 
#right order
test = test[~test.isin(test_missing_trap)].dropna(how='all')

#dropping NaN columns from original merge attempt, and adding new columns based on parent trap id
test_missing_trap.dropna(axis=1, inplace=True)
test_missing_trap = pd.merge(test_missing_trap, missing_traps[["Trap", 'Trap_Mos_Mean', 'Trap_Mos_Median', 
                                                               'Trap_Mos_25percent','Trap_Mos_75percent']],
                             how="left", left_on=["Parent_Trap"], right_on=["Trap"])

#dropping added trap_y column, and renaming trap_x to original trap
test_missing_trap.drop("Trap_y", axis=1, inplace=True)
test_missing_trap.rename(columns={'Trap_x':'Trap'}, inplace=True)

#concatenating original missing trap data for sattelite traps on test data
test = pd.concat([test, test_missing_trap], ignore_index=True, axis=0)


#inputting base numbers into NaN values based on number of mosquitos across all trapsfollowing merge
test["Trap_Mos_Mean"].fillna(value=train["NumMosquitos"].mean(), inplace=True)
test["Trap_Mos_Median"].fillna(value=train["NumMosquitos"].median(), inplace=True)
test["Trap_Mos_25percent"].fillna(value=train["NumMosquitos"].quantile(q=.25), inplace=True)
test["Trap_Mos_75percent"].fillna(value=train["NumMosquitos"].quantile(q=.75), inplace=True)

test.shape

In [38]:
#inputting 'Trap_Species_Mos_Mean', 'Trap_Species_Mos_Median', 'Trap_Species_Mos_25percent', 'Trap_Species_Mos_75percent', 

#isolating and merging Trap_Species_WNV_Prob variable from train into test set
trap_mos_num = train.groupby(['Trap',"Species", 'Trap_Species_Mos_Mean', 'Trap_Species_Mos_Median', 
                                   'Trap_Species_Mos_25percent', 'Trap_Species_Mos_75percent']).size()
trap_mos_num = pd.DataFrame(trap_mos_num)
trap_mos_num.reset_index(inplace=True)
trap_mos_num

test = pd.merge(test, trap_mos_num[["Trap","Species", 'Trap_Species_Mos_Mean', 'Trap_Species_Mos_Median', 
                                   'Trap_Species_Mos_25percent', 'Trap_Species_Mos_75percent']],how="left", 
                left_on=["Trap",'Species'], right_on=["Trap",'Species'])

#Lots of NaNs left here (63202) - how to inpute? Values are NaN if species was never see in trap before.  
#Perhaps take an average of the Mos_WNV_Prob and Trap_WNV Prob?
test["Trap_Species_Mos_Mean"].fillna((test["Species_Mos_Mean"] + test["Trap_Mos_Mean"])/2, inplace=True)
test["Trap_Species_Mos_Median"].fillna((test["Species_Mos_Median"] + test["Trap_Mos_Median"])/2, inplace=True)
test["Trap_Species_Mos_25percent"].fillna((test["Species_Mos_25percent"] + test["Trap_Mos_25percent"])/2, inplace=True)
test["Trap_Species_Mos_75percent"].fillna((test["Species_Mos_75percent"] + test["Trap_Mos_75percent"])/2, inplace=True)

test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,...,Species_Mos_75percent,Species_Obs_Proportion,Trap_Mos_Mean,Trap_Mos_Median,Trap_Mos_25percent,Trap_Mos_75percent,Trap_Species_Mos_Mean,Trap_Species_Mos_Median,Trap_Species_Mos_25percent,Trap_Species_Mos_75percent
0,1.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,14.0,0.490732,25.238095,10.0,3.0,24.0,30.650794,14.0,4.0,29.0
1,2.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,8.0,0.173513,25.238095,10.0,3.0,24.0,26.466667,7.0,3.0,17.0
2,3.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,10.0,0.330801,25.238095,10.0,3.0,24.0,15.837838,7.0,3.0,32.0
3,4.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,2.0,0.001074,25.238095,10.0,3.0,24.0,1.000000,1.0,1.0,1.0
4,5.0,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41.0,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,...,2.0,0.003777,25.238095,10.0,3.0,24.0,1.000000,1.0,1.0,1.0


In [9]:
train.columns

Index([u'Date', u'Species', u'Trap', u'Latitude', u'Longitude',
       u'NumMosquitos', u'WnvPresent', u'Year', u'Parent_Trap',
       u'Is_Satellite', u'Mos_WNV_Prob', u'Trap_Ever_Wnv',
       u'Num_Years_Trap_WNV_Detection', u'Max_One_Year_Trap_WNV_Detections',
       u'Total_Train_Trap_WNV_Detections', u'Trap_WNV_Prob', u'NumMos_3ob_avg',
       u'Trap_Species_WNV_Prob', u'Trap_Species_Mos_Mean',
       u'Trap_Species_Mos_Median', u'Trap_Species_Mos_25percent',
       u'Trap_Species_Mos_75percent', u'Trap_Mos_Mean', u'Trap_Mos_Median',
       u'Trap_Mos_25percent', u'Trap_Mos_75percent', u'Species_Mos_Mean',
       u'Species_Mos_Median', u'Species_Mos_25percent',
       u'Species_Mos_75percent', u'Species_Obs_Proportion'],
      dtype='object')

In [39]:
#preparing test data for scaling
copy_test = test.copy()
copy_test.drop(['Address', 'Species', 'Block', 'Street', 'Trap','AddressNumberAndStreet',
                 'AddressAccuracy', 'Parent_Trap'], axis=1, inplace=True)

In [42]:
#scaling test set
scaled_test =  StandardScaler().fit_transform(copy_test.iloc[:,2:])

scaled_test = pd.DataFrame(scaled_test, columns = copy_test.columns[2:])

scaled_test["Date"] = test["Date"]
scaled_test["Id"] = test["Id"]

scaled_test.head()

,Latitude,Longitude,Mos_WNV_Prob,Num_Years_Trap_WNV_Detection,Max_One_Year_Trap_WNV_Detections,Total_Train_Trap_WNV_Detections,Trap_WNV_Prob,Trap_Species_WNV_Prob,Species_Mos_Mean,Species_Mos_Median,...,Trap_Mos_Mean,Trap_Mos_Median,Trap_Mos_25percent,Trap_Mos_75percent,Trap_Species_Mos_Mean,Trap_Species_Mos_Median,Trap_Species_Mos_25percent,Trap_Species_Mos_75percent,Date,Id
0,0.987891,-1.330045,0.885971,2.285529,1.597233,2.448327,1.176137,2.129113,0.974530,0.850425,...,0.699689,1.006187,0.995285,0.640559,1.080622,1.176920,0.539881,0.924092,2008-06-11,1.0
1,0.987891,-1.330045,-0.255449,2.285529,1.597233,2.448327,1.176137,-0.704308,-0.009351,-0.130088,...,0.699689,1.006187,0.995285,0.640559,0.864934,0.326036,0.232254,0.339315,2008-06-11,2.0
2,0.987891,-1.330045,1.933189,2.285529,1.597233,2.448327,1.176137,2.512008,1.704737,0.360168,...,0.699689,1.006187,0.995285,0.640559,0.317026,0.326036,0.232254,1.070286,2008-06-11,3.0
3,0.987891,-1.330045,-0.846322,2.285529,1.597233,2.448327,1.176137,-0.704308,-1.060600,-1.110601,...,0.699689,1.006187,0.995285,0.640559,-0.447852,-0.403293,-0.383002,-0.440387,2008-06-11,4.0
4,0.987891,-1.330045,-0.846322,2.285529,1.597233,2.448327,1.176137,-0.704308,-0.980347,-1.110601,...,0.699689,1.006187,0.995285,0.640559,-0.447852,-0.403293,-0.383002,-0.440387,2008-06-11,5.0


In [43]:
#reading in cleaned weather data
weather = pd.read_csv("../assets/clean_weather.csv", index_col=0)
weather.reset_index(inplace=True, drop=True)
weather["Date"] = pd.to_datetime(weather["Date"], infer_datetime_format=True)

#dropping Depart, Wetbulb, Heat, Cool, Sunrise, Sunset, SeaLevel, ResultSpeed from weather, as many of these columns
#seem either duplicative or unimportant for purposes of this analysis
#after conducting EDA of weather, I'm also dropping Tmax, Tmin, and ResultDir

weather.drop(["Tmax", "Tmin", "ResultDir","Depart", "WetBulb", "Heat", "Cool", "Sunrise", "Sunset", "SeaLevel", 
              "ResultSpeed"], axis=1, inplace=True)

weather.head()

,Date,Tavg,DewPoint,PrecipTotal,StnPressure,AvgSpeed,Precip_7d_avg,wind_abv_1std
0,2007-05-01,67.0,51,0.000,29.10,9.2,0.00000,0.0
1,2007-05-02,51.0,42,0.000,29.38,13.4,0.00000,1.0
2,2007-05-03,56.0,40,0.000,29.39,11.9,0.00000,1.0
3,2007-05-04,58.0,41,0.001,29.31,10.8,0.00025,0.0
4,2007-05-05,60.0,38,0.001,29.40,12.0,0.00040,1.0


In [44]:
#scaling weather data before merging with train data, to take advantage of ALL weather data when scaling, rather than
#just observations that would appear in the train set
scaled_weather = StandardScaler().fit_transform(weather.iloc[:,1:])

scaled_weather = pd.DataFrame(scaled_weather, columns = weather.columns[1:])

#adding date values into scaled dataframe
scaled_weather["Date"] = weather["Date"]
scaled_weather["wind_abv_1std"] = weather["wind_abv_1std"]

scaled_weather.head()

,Tavg,DewPoint,PrecipTotal,StnPressure,AvgSpeed,Precip_7d_avg,wind_abv_1std,Date
0,0.037433,-0.222912,-0.319916,-0.987494,0.197482,-0.755462,0.0,2007-05-01
1,-1.485236,-1.066664,-0.319916,0.806746,1.515681,-0.755462,1.0,2007-05-02
2,-1.009402,-1.254164,-0.319916,0.870826,1.044896,-0.755462,1.0,2007-05-03
3,-0.819068,-1.160414,-0.317540,0.358186,0.699653,-0.754059,0.0,2007-05-04
4,-0.628734,-1.441665,-0.317540,0.934906,1.076281,-0.753217,1.0,2007-05-05


In [45]:
combined_test = pd.merge(scaled_weather, scaled_test, how= "inner", left_on='Date', right_on='Date')


In [46]:
#dropping Date and id from
X = combined_test.copy()

X.drop(["Date", "Id"], axis=1, inplace=True)

In [47]:
#saving new cleaned train set to assets folder
X.to_csv("../assets/clean_transform_test_2.csv")